<a href="https://colab.research.google.com/github/Yajat-Parikh/Melt-Pool-Dimension-Estimation/blob/main/Model%20training%20(train%20data)%20and%20evaluation%20(test%20data).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb
import joblib

In [11]:
df=pd.read_csv("/Processed_Training_Data.csv")

In [12]:
df.head()

,Powder Material,Power (W),Travel Velocity (mm/s),Melt Pool Depth (mm),Melt Pool Width (mm),Melt Pool Length (mm),beamD (mm),Fe (%),Ni (%),Cr (%),...,Mn (%),Si (%),Cu (%),Nb (%),W (%),V (%),Mg (%),Zn (%),Y (%),Nd (%)
0,IN625,160.0,2800.0,0.04,0.09200,0.524400,50.0,0,58,22,...,0.0,0,5,0,0,0,0,0,0,0
1,IN625,160.0,2800.0,0.04,0.09150,0.521550,50.0,0,58,22,...,0.0,0,5,0,0,0,0,0,0,0
2,IN625,160.0,2800.0,0.04,0.09152,0.521664,50.0,0,58,22,...,0.0,0,5,0,0,0,0,0,0,0
3,IN625,160.0,2800.0,0.04,0.09250,0.527250,50.0,0,58,22,...,0.0,0,5,0,0,0,0,0,0,0
4,IN625,160.0,2800.0,0.04,0.09300,0.530100,50.0,0,58,22,...,0.0,0,5,0,0,0,0,0,0,0


In [13]:
df.dropna().shape

(218, 25)

In [14]:
df['Powder Material'].value_counts()

,count
Powder Material,
SS17-4PH,70
Ti-6Al-4V,55
SS316L,32
IN625,29
AlSi10Mg,13
Invar36,9
TiC/Inconel 718,8
IN718,2


In [17]:

# Prepare data
features = ['Power (W)', 'Travel Velocity (mm/s)', 'beamD (mm)', 'Fe (%)', 'Ni (%)',
            'Cr (%)', 'Co (%)', 'Al (%)', 'Ti (%)', 'Mo (%)', 'C (%)', 'Mn (%)',
            'Si (%)', 'Cu (%)', 'Nb (%)', 'W (%)', 'V (%)', 'Mg (%)', 'Zn (%)',
            'Y (%)', 'Nd (%)']

X = df.dropna()[features]
targets = {
    'Length': df.dropna()['Melt Pool Length (mm)'],
    'Width': df.dropna()['Melt Pool Width (mm)'],
    'Depth': df.dropna()['Melt Pool Depth (mm)']
}

X_train, X_test, y_train_dict, y_test_dict = train_test_split(
    X,
    pd.concat(targets.values(), axis=1),
    test_size=0.1,
    random_state=42
)

def train_and_eval(model_class, param_grid, name):
    for i, target in enumerate(targets):
        y_train = y_train_dict.iloc[:, i]
        y_test = y_test_dict.iloc[:, i]

        grid = GridSearchCV(model_class(random_state=42), param_grid, cv=5)
        grid.fit(X_train, y_train)

        y_pred = grid.predict(X_test)
        print(f"{name} - {target}")
        print("MSE:", mean_squared_error(y_test, y_pred))
        print("R²:", r2_score(y_test, y_pred))
        print()

# Random Forest
rf_params = {
    'n_estimators': [50, 100],
    'max_depth': [5, 10],
    'min_samples_split': [2, 5],
    'max_features': ['sqrt']
}
train_and_eval(RandomForestRegressor, rf_params, "Random Forest")

# Gradient Boosting
gb_params = {
    'n_estimators': [100],
    'learning_rate': [0.05, 0.1],
    'max_depth': [3, 4],
    'subsample': [0.8],
    'max_features': ['sqrt']
}
train_and_eval(GradientBoostingRegressor, gb_params, "Gradient Boosting")

# XGBoost
xgb_params = {
    'n_estimators': [100],
    'learning_rate': [0.05, 0.1],
    'max_depth': [3, 5],
    'subsample': [0.8],
    'colsample_bytree': [0.8],
    'reg_alpha': [0, 0.5],
    'reg_lambda': [1]
}
train_and_eval(xgb.XGBRegressor, xgb_params, "XGBoost")



Random Forest - Length
MSE: 0.007109614751068621
R²: 0.9844943739729394

Random Forest - Width
MSE: 0.0003601547455419854
R²: 0.9595602220849166

Random Forest - Depth
MSE: 0.0005133356057621517
R²: 0.7076060738927874

Gradient Boosting - Length
MSE: 0.006293288782026726
R²: 0.9862747299606163

Gradient Boosting - Width
MSE: 0.00043532928503569457
R²: 0.9511192901809946

Gradient Boosting - Depth
MSE: 0.0004266473188990661
R²: 0.7569833784453771

XGBoost - Length
MSE: 0.007962097025945703
R²: 0.9826351633389235

XGBoost - Width
MSE: 0.00029204628274650114
R²: 0.9672077434453314

XGBoost - Depth
MSE: 0.00042615415756202987
R²: 0.7572642811879917



In [19]:
# Save trained models
def save_best_models():
    models = [
        ('random_forest', RandomForestRegressor, rf_params),
        ('gradient_boosting', GradientBoostingRegressor, gb_params),
        ('xgboost', xgb.XGBRegressor, xgb_params)
    ]

    for model_name, model_class, param_grid in models:
        for i, target in enumerate(targets):
            y_train = y_train_dict.iloc[:, i]
            grid = GridSearchCV(model_class(random_state=42), param_grid, cv=5)
            grid.fit(X_train, y_train)
            filename = f"{model_name}_{target.lower()}.pkl"
            joblib.dump(grid.best_estimator_, filename)

save_best_models()


In [20]:
df_test=pd.read_csv("/Processed_Testing_data.csv")

In [22]:
df_test.columns

Index(['Powder Material', 'Power (W)', 'Travel Velocity (mm/s)',
       'Melt Pool Depth (mm)', 'Melt Pool Width (mm)', 'Melt Pool Length (mm)',
       'beamD (mm)', 'Fe (%)', 'Ni (%)', 'Cr (%)', 'Co (%)', 'Al (%)',
       'Ti (%)', 'Mo (%)', 'C (%)', 'Mn (%)', 'Si (%)', 'Cu (%)', 'Nb (%)',
       'W (%)', 'V (%)', 'Mg (%)', 'Zn (%)', 'Y (%)', 'Nd (%)'],
      dtype='object')

In [25]:
# Prepare test data
X_test_new = df_test[features]

# Ground truth columns
y_true = {
    'Length': df_test['Melt Pool Length (mm)'],
    'Width': df_test['Melt Pool Width (mm)'],
    'Depth': df_test['Melt Pool Depth (mm)']
}

# Model types and targets
model_types = ['random_forest', 'gradient_boosting', 'xgboost']
target_names = ['Length', 'Width', 'Depth']

# Predict and print scores
for model in model_types:
    for target in target_names:
        model_path = f"/content/Saved Models/{model}_{target.lower()}.pkl"
        loaded_model = joblib.load(model_path)
        y_pred = loaded_model.predict(X_test_new)

        # Print performance
        if target in y_true:
            y_actual = y_true[target]
            mse = mean_squared_error(y_actual, y_pred)
            r2 = r2_score(y_actual, y_pred)
            print(f"{model} - {target}")
            print("MSE:", round(mse, 4))
            print("R² :", round(r2, 4))
            print()

random_forest - Length
MSE: 0.0005
R² : -0.5096

random_forest - Width
MSE: 0.0003
R² : -4.977

random_forest - Depth
MSE: 0.0001
R² : -0.9786

gradient_boosting - Length
MSE: 0.0059
R² : -15.8488

gradient_boosting - Width
MSE: 0.0006
R² : -11.7872

gradient_boosting - Depth
MSE: 0.0007
R² : -11.1086

xgboost - Length
MSE: 0.0062
R² : -16.7555

xgboost - Width
MSE: 0.0003
R² : -4.5365

xgboost - Depth
MSE: 0.0002
R² : -2.7135

